In [1]:
pip install loguru

Note: you may need to restart the kernel to use updated packages.


In [6]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
ua = UserAgent()
import requests
import pandas as pd
import lxml
import threading
import os
import time
import math
from loguru import logger
import re

In [7]:
def get_headers():
    headers = {'User-Agent': ua.random}
    return headers

logger.add('file_{time}.log', format = '{name} {message}', level = 'DEBUG', rotation = '5 MB')
urlsBooks = []
allBooks = []
page_num_all = []
filteredBooks = pd.DataFrame(columns = ['id', 'Name', 'Author', 'Publisher', 'Rating', 'OldPrice', 'NewPrice', 'path',
                                        'genre', 'yearOfIssue', 'numberOfPage', 'ageLimit'])

## Получение кол-ва страниц на сайте
link = 'https://www.chitai-gorod.ru/catalog/books-18030'
page = requests.get(link, headers = get_headers())
soup = BeautifulSoup(page.text, 'lxml')
pagination = soup.find('div', class_ = 'pagination')
page_num_all = pagination.find_all('span', class_ = 'pagination__text')
count_of_page = page_num_all[-1].get_text(strip = True)
count_of_page = int(count_of_page) + 1
    
#Кол-во страниц на один поток
range_links = math.ceil(count_of_page / os.cpu_count())

In [ ]:
#Получение ссылок на книги со страницы
@logger.catch
def links_get(n):
    i = 0
    j = 0
    n = n * range_links
    k = n + range_links
    if n == 0:
        n = 1
        
    for x in range(1,5):
        link_page = 'https://www.chitai-gorod.ru/catalog/books-18030?page=' + str(x)
        page = requests.get(link_page, headers = get_headers())
        
        while (page.status_code != 200) & (i != 5): 
            time.sleep(10)
            page = requests.get(link_page, headers = get_headers())
            i +=1
        
        if page.status_code != 200:
            logger.debug(f'The page {link_page} was not processed with status code {page}')
            continue
            
        soup = BeautifulSoup(page.text, 'lxml')
        product_list = soup.find('div', class_ = 'products-list')
        
        while (product_list is None) & (j != 5):
            time.sleep(10)
            page = requests.get(link_page, headers = get_headers())
            soup = BeautifulSoup(page.text, 'lxml')
            product_list = soup.find('div', class_ = 'products-list')
            j += 1
        
        if product_list is None:
            logger.debug(f'The page {link_page} is None')
            continue
            
        allBooks = product_list.find_all('article')
        
        for i in allBooks:
            links = 'https://www.chitai-gorod.ru' + i.find(class_ = 'product-card__picture product-card__row')['href']
            urlsBooks.append(links)

def run_threads():
    threads = []
    n = 0
    
    for x in range(os.cpu_count()):   
        t = threading.Thread(target = links_get, args = (n,))
        n += 1
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

if __name__ == '__main__':
        run_threads()

In [ ]:
range_book_url = math.ceil(len(urlsBooks) / os.cpu_count())

#Фун-ия для форматирования строк
def reformat(x, mode = 'default'):
    if mode == 'default':
        x = str(x.contents[0])
        a = re.sub(r'[\n]+', '', x)
        a.strip() 
    else:
        x = str(x[0])
        a = re.sub(r"[\s\n]+", ' ', x)
        if mode == 'genre':
            a = re.split(r"\s+(?=[А-ЯЁ])", a)
            a = a[-1]
    return a

#Получение информации о книге с ее страницы
@logger.catch
def book_info(n):
    m = 0
    n = n * range_book_url
    k = n + range_book_url
    
    urlsBooks_info = urlsBooks[n:k:]
    
    for x in urlsBooks_info:
        book = []
        book_url = x
        page = requests.get(book_url, headers = get_headers())
        
        while (page.status_code != 200) & (m != 5): 
            time.sleep(10)
            page = requests.get(book_url, headers = get_headers())
            m +=1
        
        if page.status_code != 200:
            logger.debug(f'The page {book_url} was not processed with status code {page}')
            continue
            
        soup = BeautifulSoup(page.text, 'lxml')
        pathAll = []
        id_tag = soup.find('span', class_='product-detail-features__item-value')
        name_tag = soup.find('h1', class_='detail-product__header-title')
        author_tag = soup.find('a', class_='product-info-authors__author')
        izd_tag = soup.find('a', class_='product-detail-features__item-value product-detail-features__item-value--link')
        rating_tag = soup.find('span', class_='product-review-range__count')
        oldPrice_tag = soup.find('span', class_='product-offer-price__old-price')
        newPrice_tag = soup.find('span', class_='product-offer-price__current product-offer-price__current--discount')
        year_tag = soup.find('span', itemprop = 'datePublished')
        numberOfPages_tag = soup.find('span', itemprop = 'numberOfPages')
        typicalAgeRange_tag = soup.find('span', itemprop = 'typicalAgeRange')
        genre_tag = soup.findAll('ul', class_='product-breadcrumbs detail-product__breadcrumbs')
        
        for i in genre_tag:
            temp = i.get_text()
            pathAll.append(temp)
        
        id = reformat(id_tag)
        book.append(id)
                
        name = reformat(name_tag)
        book.append(name)
        
        if author_tag is not None:
            author = reformat(author_tag)
        else:
            author = 'Без автора'    
        book.append(author)
        
        if izd_tag is not None:
            izd = reformat(izd_tag)
        else:
            izd = 'Без издательства'
        book.append(izd)
        
        if rating_tag is not None:
            rating = reformat(rating_tag)
        else:
            rating = 'Без рейтинга'    
        book.append(rating)
        
        if oldPrice_tag is not None:
            oldPrice = reformat(oldPrice_tag)
        else:
            oldPrice = 'Без цены'
        book.append(oldPrice)
        
        if newPrice_tag is not None:
            newPrice = reformat(newPrice_tag)
        else:
            newPrice = 'Без цены'
        book.append(newPrice)

        path = reformat(pathAll, 'path')
        book.append(path)
        
        genre = reformat(pathAll, 'genre')
        book.append(genre)
        
        if year_tag is not None:
            year = reformat(year_tag)
        else:
            year = 'Без года'
        book.append(year)
        
        if numberOfPages_tag is not None:
            numberOfPages = reformat(numberOfPages_tag)
        else:
            numberOfPages = 'Без кол-ва стр'
        book.append(numberOfPages)
        
        if typicalAgeRange_tag is not None:
            typicalAgeRange = reformat(typicalAgeRange_tag)
        else:
            typicalAgeRange = 'Без возрастных ограничений'
        book.append(typicalAgeRange)

        filteredBooks.loc[len(filteredBooks)] = book

def run_threads_book():    
    threads = []
    n = 0 
    
    for x in range(os.cpu_count()):
        t = threading.Thread(target = book_info, args = (n,))
        t.start()
        threads.append(t)
        n += 1
        
    for t in threads:
        t.join()

if __name__ == '__main__':
    
    run_threads_book()

filteredBooks.to_csv('parserBook.csv')

In [ ]:
filteredBooks